# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic
- On Policy

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [ ] Experience Replay for Random Sampling
- [ ] Importance Sampling
    
### Deterministic Policy Gradient
- [ ] DDPG
- [ ] MADDPG

### Stability and Reducing Variance
- [x] Gradient clipping
- [ ] Normalized Reward/Advantage
- [ ] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Synchronous Training (A2C)
- [x] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb

## TODO:

- Research on '_bootstrap_' instead of end-reward
- Add global step
- Think about adding discont to advantage
- Normalize reward?
- Record method in network

In [1]:
!rm -rf logs/B4R4_nonzero_A3C_2_11/ model/B4R4_nonzero_A3C_2_11

In [2]:
TRAIN_NAME='B4R4_nonzero_A3C_2_11'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.25 # gpu capacity in percentage

In [3]:
import os

import signal
import threading
import multiprocessing

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random
import math

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder_v2 as one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import Experience_buffer, discount_rewards

from network.ActorCritic2 import ActorCritic as AC

%load_ext autoreload
%autoreload 2

## Hyperparameters

In [4]:
# Replay Variables
total_episodes= 120000
max_ep = 150
update_frequency = 32
entropy_beta = 0.01

# Saving Related
save_network_frequency = 1200
save_stat_frequency = 128
moving_average_step = 128

# Training Variables
decay_lr = False
lr_a = 5e-5
lr_c = 4e-4
lr_a_gamma = 0.9995
lr_c_gamma = 0.9995
lr_a_final = 1e-5
lr_c_final = 1e-4
lr_a_decay_step = (int) (math.log(lr_a_final/lr_a) / math.log(lr_a_gamma))
lr_c_decay_step = (int) (math.log(lr_c_final/lr_c) / math.log(lr_c_gamma))

gamma = 0.99 # discount_factor

# Env Settings
MAP_SIZE = 20
VISION_RANGE = 9 # What decide the network size !!!
VISION_dX, VISION_dY = 2*VISION_RANGE+1, 2*VISION_RANGE+1
NCHANNEL = 11
in_size = [None,VISION_dX,VISION_dY,NCHANNEL]
NENV = (int) (multiprocessing.cpu_count())

# Asynch Settings
global_scope = 'global'

## Environment Setting

In [ ]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [ ]:
action_space = 5
n_agent = 4

## A3C Network Structure

![Network Structure](https://cdn-images-1.medium.com/max/1600/1*YtnGhtSAMnnHSL8PvS7t_w.png)

- Network is given in network.ActorCritic

## Environments

![Environment Interaction Diagram](https://cdn-images-1.medium.com/max/1600/1*Hzql_1t0-wwDxiz0C97AcQ.png)

In [ ]:
global_rewards = MA(moving_average_step)
global_ep_rewards = MA(moving_average_step)
global_length = MA(moving_average_step)
global_succeed = MA(moving_average_step)
global_episodes = 0

# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
progbar = tf.keras.utils.Progbar(total_episodes,interval=1)

## Worker

In [ ]:
class Worker(object):
    def __init__(self, name, globalAC, sess, global_step=0):
        # Initialize Environment worker
        self.env = gym.make("cap-v0").unwrapped
        self.name = name
        
        # Create AC Network for Worker
        self.AC = AC(in_size=in_size,
                     action_size=action_space,
                     num_agent=n_agent,
                     decay_lr=decay_lr,
                     lr_actor=lr_a,
                     lr_critic=lr_c,
                     grad_clip_norm=0,
                     scope=self.name,
                     global_step=global_step,
                     initial_step=0,
                     entropy_beta = 0.01,
                     sess=sess,
                     global_network=global_ac)
        
        self.sess=sess
        
    def get_action(self, s):
        cr_list = []
        mask_list = []
        is_alive = [ag.isAlive for ag in self.env.get_team_blue]
        for idx in range(n_agent):
            cr_list.append(np.concatenate((s[idx:idx+1],s[:idx],s[idx+1:])))
            mask_list.append([1]+(is_alive[:idx]+is_alive[idx+1:])*1)
        feed_dict = {self.AC.state_input : s,
                     self.AC.critic_state_input : np.array(cr_list),
                     self.AC.mask : mask_list}
        a1, v1, _ = self.AC.run_network(feed_dict)
        
        return a1, v1, _
        
    def work(self, saver, writer):
        global global_rewards, global_ep_rewards, global_episodes, global_length, global_succeed
        total_step = 1
                
        # loop
        with self.sess.as_default(), self.sess.graph.as_default():
            while not coord.should_stop() and global_episodes < total_episodes:
                s0 = self.env.reset(map_size=MAP_SIZE, policy_red=policy.zeros.PolicyGen(self.env.get_map, self.env.get_team_red))
                #s0 = one_hot_encoder(s0, self.env.get_team_blue, VISION_RANGE)
                s0 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                
                # parameters 
                ep_r = 0 # Episodic Reward
                prev_r = 0
                was_alive = [ag.isAlive for ag in self.env.get_team_blue]

                indv_history = [ [] for _ in range(len(self.env.get_team_blue)) ]
                
                # Bootstrap
                a1, v1, _ = self.get_action(s0)
                for step in range(max_ep+1):
                    a, v0 = a1, v1
                    
                    s1, rc, d, _ = self.env.step(a)
                    #s1 = one_hot_encoder(s1, self.env.get_team_blue, VISION_RANGE)
                    s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                    is_alive = [ag.isAlive for ag in self.env.get_team_blue]
                    r = (rc - prev_r-1)
                    #d = (True if d or step == max_ep else False)
                    if step == max_ep and d == False:
                        r = -100
                        rc = -100
                        d = True

                    r /= 100.0
                    ep_r += r

                    if d:
                        v1 = [0.0 for _ in range(len(self.env.get_team_blue))]
                    else:
                        a1, v1, _ = self.get_action(s1)

                    # push to buffer
                    for idx, agent in enumerate(self.env.get_team_blue):
                        if was_alive[idx]:
                            indv_history[idx].append([s0[idx],
                                                      a[idx],
                                                      r,
                                                      v0[idx],
                                                      np.concatenate((s0[idx:idx+1],s0[:idx],s0[idx+1:])).tolist(),
                                                      [1]+(is_alive[:idx]+is_alive[idx+1:])*1
                                                     ])

                    if total_step % update_frequency == 0 or d:
                        aloss, closs, etrpy, feed_dict = self.train(indv_history, sess, v1)
                        indv_history = [[] for _ in range(len(self.env.get_team_blue))]

                    # Iteration
                    prev_r = rc
                    was_alive = is_alive
                    s0=s1
                    total_step += 1

                    if d:
                        break
                        
                global_ep_rewards.append(ep_r)
                global_rewards.append(rc)
                global_length.append(step)
                global_succeed.append(self.env.blue_win)
                global_episodes += 1
                self.sess.run(global_step_next)
                progbar.update(global_episodes)
                if global_episodes % save_stat_frequency == 0 and global_episodes != 0:
                    summary_ = sess.run(merged_summary_op, feed_dict)
                    summary = tf.Summary()
                    summary.value.add(tag='Records/mean_reward', simple_value=global_rewards())
                    summary.value.add(tag='Records/mean_length', simple_value=global_length())
                    summary.value.add(tag='Records/mean_succeed', simple_value=global_succeed())
                    summary.value.add(tag='Records/mean_episode_reward', simple_value=global_ep_rewards())
                    summary.value.add(tag='summary/Entropy', simple_value=etrpy)
                    summary.value.add(tag='summary/actor_loss', simple_value=aloss)
                    summary.value.add(tag='summary/critic_loss', simple_value=closs)
                    writer.add_summary(summary,global_episodes)
                    writer.add_summary(summary_,global_episodes)

                    writer.flush()
                if global_episodes % save_network_frequency == 0 and global_episodes != 0:
                    saver.save(self.sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)
                '''if global_episode == 45000:
                    MAP_SIZE=20'''

    def train(self, indv_buffer, sess, bootstrap=0.0):
        buffer_s, buffer_a, buffer_tdtarget, buffer_adv, buffer_cs, buffer_msk = [], [], [], [], [], []
        for idx, buffer in enumerate(indv_buffer):
            if len(buffer) == 0:
                continue
            _history = np.array(buffer)
            observations = _history[:,0]
            actions = _history[:,1].astype(np.int32)
            rewards = _history[:,2]
            values = _history[:,3]
            cr_observations = _history[:,4]
            masks = _history[:,5]
            
            value_ext = np.append(values, [bootstrap[idx]])
            td_target = rewards + gamma * value_ext[1:]
            advantages = rewards + gamma * value_ext[1:] - value_ext[:-1]
            advantages = discount_rewards(advantages,gamma)
            
            buffer_s.extend(observations.tolist())
            buffer_a.extend(actions.tolist())
            buffer_tdtarget.extend(td_target.tolist())
            buffer_adv.extend(advantages.tolist())
            buffer_cs.extend(cr_observations.tolist())
            buffer_msk.extend(masks.tolist())


        buffer_s, buffer_a, buffer_tdtarget, buffer_adv = np.stack(buffer_s), np.array(buffer_a), np.array(buffer_tdtarget), np.array(buffer_adv)
        buffer_cs = np.array(buffer_cs)
        buffer_msk = np.array(buffer_msk)
        feed_dict = {
            self.AC.state_input : buffer_s,
            self.AC.action_ : buffer_a,
            self.AC.td_target_ : buffer_tdtarget,
            self.AC.advantage_ : buffer_adv,
            self.AC.critic_state_input : buffer_cs,
            self.AC.mask : buffer_msk
        }

        # Update Buffer
        aloss, closs, etrpy = self.AC.update_global(feed_dict)

        # get global parameters to local ActorCritic 
        self.AC.pull_global()
        
        return aloss, closs, etrpy, feed_dict
    

## Run

In [ ]:
with tf.device("/cpu:0"):
    # Global Network
    global_step = tf.Variable(0, trainable=False, name='global_step')
    global_step_next = tf.assign_add(global_step, 1)
    global_ac = AC(in_size=in_size, action_size=action_space, num_agent=n_agent, scope=global_scope, sess=sess, global_step=global_step)
    
    # Local workers
    workers = []
    # loop for each workers
    for idx in range(NENV):
        name = 'W_%i' % idx
        workers.append(Worker(name, global_ac, sess, global_step=global_step))
        print(f'worker: {name} initiated')
    saver = tf.train.Saver(max_to_keep=3)
    writer = tf.summary.FileWriter(LOG_PATH, sess.graph)
    
ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")
    
coord = tf.train.Coordinator()
worker_threads = []
global_episodes = sess.run(global_step)

saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)

# Summarize
for var in tf.trainable_variables(scope=global_scope):
    tf.summary.histogram(var.name, var)
merged_summary_op = tf.summary.merge_all()

for worker in workers:
    job = lambda: worker.work(saver, writer)
    t = threading.Thread(target=job)
    t.start()
    worker_threads.append(t)
coord.join(worker_threads)


worker: W_0 initiated
worker: W_1 initiated
worker: W_2 initiated
worker: W_3 initiated
worker: W_4 initiated
worker: W_5 initiated
worker: W_6 initiated
worker: W_7 initiated
worker: W_8 initiated
worker: W_9 initiated
worker: W_10 initiated
worker: W_11 initiated
worker: W_12 initiated
worker: W_13 initiated
worker: W_14 initiated
worker: W_15 initiated
worker: W_16 initiated
worker: W_17 initiated
worker: W_18 initiated
worker: W_19 initiated
worker: W_20 initiated
worker: W_21 initiated
worker: W_22 initiated
worker: W_23 initiated
Initialized Variables
INFO:tensorflow:Summary name global/actor/conv1/weights:0 is illegal; using global/actor/conv1/weights_0 instead.
INFO:tensorflow:Summary name global/actor/conv1/biases:0 is illegal; using global/actor/conv1/biases_0 instead.
INFO:tensorflow:Summary name global/actor/conv2/weights:0 is illegal; using global/actor/conv2/weights_0 instead.
INFO:tensorflow:Summary name global/actor/conv2/biases:0 is illegal; using global/actor/conv2/bi